# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [2]:
import tensorflow as tf
import unicodedata
import re
import numpy as np

![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [2]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

In [3]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [4]:
print(data_examples[10])

Go on.	Mach weiter.	CC-BY 2.0 (France) Attribution: tatoeba.org #2230774 (CK) & #6625701 (Felixjp)



In [5]:
sentences =[]
for i in data_examples:
    sentences.append(i.split("	"))

sentences[5]

['Fire!',
 'Feuer!',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #1958697 (Tamy)\n']

In [6]:
english_sentences=[]
german_sentences =[]
for s in sentences:
    english_sentences.append(s[0])
    german_sentences.append(s[1])
   

index=10000

print(english_sentences[index], german_sentences[index])

Get off my lawn. Gehen Sie von meinem Rasen!


In [7]:
start_token = '<start>'
end_token = '<end>'

for es in english_sentences:
    es=preprocess_sentence(es)
    
for gs in german_sentences:
    gs=start_token+preprocess_sentence(gs)+end_token


In [8]:
print(german_sentences[20])

Iss auf.


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters='')

In [10]:
import json

tokenizer.fit_on_texts(german_sentences)
config = tokenizer.get_config()
word_index = json.loads(config['word_index'])
index_words  = json.loads(config['index_word'])

In [11]:
num_samples =5
inx = np.random.choice(len(english_sentences),num_samples, replace=False)
print(inx)

[ 9997 11137 18099  4562  8469]


In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences=tokenizer.texts_to_sequences(german_sentences)
padded = pad_sequences(sequences, padding='post',value=0)


## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [13]:
emb_layer = tf.keras.models.load_model('./models/tf2-preview_nnlm-en-dim128_1')


In [14]:
# Test the layer
emb_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape



TensorShape([7, 128])

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [46]:
sequences = tokenizer.texts_to_sequences(german_sentences)

In [16]:
from sklearn.model_selection import train_test_split

x_english_train, x_english_test = train_test_split(english_sentences, test_size=0.2)
x_german_train, x_german_test = train_test_split(sequences, test_size=0.2)


x_english_train = np.array(x_english_train)
x_english_test = np.array(x_english_test)
x_german_train = np.array(x_german_train)
x_german_test = np.array(x_german_test)

trainDataset = tf.data.Dataset.from_tensor_slices((x_english_train,x_german_train))
validationDataset = tf.data.Dataset.from_tensor_slices((x_english_test,x_german_test ))


print(trainDataset.element_spec)
print(validationDataset.element_spec)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
def split(x,y):
    inp =[]
    inp.append(tf.strings.split(x,''))
    inp.append(y)
    return inp[0],inp[1]

trainDataset=trainDataset.map(split)
validationDataset=validationDataset.map(split)

In [49]:
def embedding_f(x,y):
    inp=[]
    inp.append(emb_layer(x))
    inp.append(y)
    return inp[0], inp[1]

trainDataset=trainDataset.map(embedding_f)
validationDataset=validationDataset.map(embedding_f)

ValueError: in converted code:

    <ipython-input-35-6aebd52c53eb>:3 embedding_f  *
        inp.append(emb_layer(x))
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/saved_model/load.py:436 _call_attribute
        return instance.__call__(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/eager/def_function.py:457 __call__
        result = self._call(*args, **kwds)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/eager/def_function.py:494 _call
        results = self._stateful_fn(*args, **kwds)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py:1822 __call__
        graph_function, args, kwargs = self._maybe_define_function(args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py:2107 _maybe_define_function
        *args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py:1650 canonicalize_function_inputs

In [50]:
trainDataset.element_spec
validationDataset.element_spec

(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.string, name=None))

In [51]:
trainDataset = trainDataset.filter(lambda x,y: False if ((tf.shape(x)[0])>13 else True))
validationDataset = validationDataset.filter(lambda x,y: False if ((tf.shape(x)[0])>13 else True))


SyntaxError: invalid syntax (<ipython-input-51-bcac88ccc2c1>, line 1)

In [52]:
def map_embedding_f(x,y):
    inp=[]
    pad=tf.pad(x, paddings=[[13-tf.shape(x)[0],0][0,0]], mode='CONSTANT')
    inp.append(pad)
    inp.append(y)
    return inp[0], inp[1]

trainDataset=trainDataset.batch(16)
validationDataset=validationDataset.batch(16)

In [53]:
trainDataset=trainDataset.batch(map_embedding_f)
validationDataset=validationDataset.map(map_embedding_f)

ValueError: Attempt to convert a value (<function map_embedding_f at 0x7f3e63310c80>) with an unsupported type (<class 'function'>) to a Tensor.

## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [4]:
from tensorflow.keras.layers import Layer
class CustomLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.learned_emb = tf.Variable(initial_value=tf.zeros(shape=(1,128),dtype=tf.float32))
        
    def call(self, inputs):
        x=self.learned_emb
        x=tf.tile(x,[tf.shape(inputs)[0],1])
        x=tf.expand_dims(x,axis=1, name=None)
        return tf.keras.layers.Concatenate(axis=1)([inputs,x])
    
myLayer = CustomLayer()

In [5]:
from tensorflow.keras.layers import Dense, Flatten, Masking, LSTM
from tensorflow.keras import Input, Model

inputs = Input(batch_shape = (None, 13, 128))
x = myLayer(inputs)
x = Masking(mask_value = 4.0)(x)
hidden_state, cell_state = LSTM(512, return_state=True, name='stateful')(x)
encoder = Model(inputs = inputs, outputs = [hidden_state, cell_state])

ValueError: too many values to unpack (expected 2)

## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [6]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout


class MyDecoder(Model):
    def __init__(self):
        super(MyDecoder,self).__init__()
        self.embedding=Embedding(len(tokenizer.word_index),128,mask_zero=True)
        self.lstm=LSTM(512,return_sequences=True, return_state=True)
        self.dense=Dense(len(tokenizer.word_index))
    def call(self,inputs, hidden_state=None,cell_state=None):
        x=self.embedding(inputs)
        if hidden_state is not None and cell_state is not None:
            x, hidden_state, cell_state=self.lstm(x, initial_state=[hidden_state, cell_state])
        else:
            x, hidden_state, cell_state=self.lstm(x)
            x=self.dense(x)
        return x, hidden_state, cell_state
        
        decoder=MyDecoder()

In [66]:
hidden, cell = Encoder(next(iter(trainDataset.take(1)))[0])
print(decoder(next(iter(trainDataset.take(1))))[1],hidden, cell)

NameError: name 'Encoder' is not defined

## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [56]:
def convert(data):
    german_inputs = tf.zeros([data.shape[0],data.shape[1], tf.int32])
    german_outputs = tf.zeros([data.shape[0],data.shape[1], tf.int32])
    german_inputs=data
    german_inputs = tf.where(german_inputs[:]==word_index.get(end_token),x=0,y=german_inputs)
    german_outputs = tf.where(german_inputs[:]==word_index.get(start_token),x=0,y=german_inputs)

    return german_inputs, german_outputs

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(0.05)
loss = tf.keras.losses.SparseCategoricalCrossEntropy(from_logits=True)
variables = Encoder.trainable_variables+ Decoder.trainable_variables

In [7]:
@tf.function
def grad_fn(data1, data, german_outputs, loss):
    with tf.GradientTape() as tape:
        x,y = Encoder(data1)
        a,b,c = Decoder(data, x, y)
        loss_value=loss(german_outputs, loss)
    return loss_value, tape.gradient(loss_value, variables)
        

In [8]:
def train_autoencoder(num_epochs, dataset, val_dataset, val_steps, grad_fn, optimizer, loss):
    train_loss_results=[]
    val_loss_results = []
    
    for epoch in range(num_epochs):
        epoch_loss_avg=tf.keras.metrics.Mean()
        val_epoch_loss_avg = tf.keras.metrics.Mean()
        
        for x,y in dataset:
            inp, out = convert(y)
            loss_value, grads = grad_fn(x,y,out, loss)
            optimizer.apply_gradients(zip(grads, variables))
            
            epoch_loss_avg(loss_value)
        for a,b in val_dataset:
            preds,_,_ = decoder(b)
            val_epoch_loss_avg(loss(out, preds))
        train_loss_results.append(epoch_loss_avg.result())
        val_loss_results.append(val_epoch_loss_avg.result())
        
        print(" Epoch :{03d}: Loss: {:.3f} : Val_loss: {:.3f}".format(epoch, epoch_loss_results(), val_epoch_loss_results()))
        
    return train_loss_results, val_loss_results

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.